# Hit prediction

In [1]:
# Import libraries
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score

In [2]:
# Load data
songs = pd.read_csv('all_time_billboard_wrap_up_cleaned_spotify.csv')
display(songs)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language,Spotify ID,Spotify Artist,Spotify Song,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,...,0.1130,-12.201,1.0,0.0371,174.578,3.0,https://api.spotify.com/v1/tracks/5dae01pKNjRQ...,audio_features,spotify:track:5dae01pKNjRQtgOeAkFzPY,0.303
1,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,...,0.1670,-13.755,1.0,0.0473,138.800,3.0,https://api.spotify.com/v1/tracks/33FdwvfiSBRb...,audio_features,spotify:track:33FdwvfiSBRbE2AQ4UehaR,0.829
2,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,en,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,...,0.1540,-14.392,1.0,0.0398,99.136,4.0,https://api.spotify.com/v1/tracks/0lO5EKoz1Rb1...,audio_features,spotify:track:0lO5EKoz1Rb1pJoPoldE4D,0.919
3,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,en,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,...,0.0849,-10.808,1.0,0.0358,115.590,4.0,https://api.spotify.com/v1/tracks/6kVfwQOc2PjV...,audio_features,spotify:track:6kVfwQOc2PjVPwbk0aEmN9,0.536
4,8,Red Foley,Chattanoogie Shoe Shine Boy,1950,7.json,Red Foley,en,7Jf323ttHKUnPylFWiaGl3,Red Foley,Chattanoogie Shoe Shine Boy - 1949 Single Version,...,0.1180,-15.925,1.0,0.0494,148.367,4.0,https://api.spotify.com/v1/tracks/7Jf323ttHKUn...,audio_features,spotify:track:7Jf323ttHKUnPylFWiaGl3,0.846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4806,95,Cole Swindell,Single Saturday Night,2021,6774.json,Cole Swindell,en,1irAliF0T8sLIOPJp6n7rU,Cole Swindell,Single Saturday Night,...,0.2260,-5.070,1.0,0.0473,81.949,4.0,https://api.spotify.com/v1/tracks/1irAliF0T8sL...,audio_features,spotify:track:1irAliF0T8sLIOPJp6n7rU,0.694
4807,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,...,0.1330,-5.623,1.0,0.0312,139.931,4.0,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,audio_features,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,0.397
4808,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,...,0.1670,-8.420,0.0,0.3240,131.988,4.0,https://api.spotify.com/v1/tracks/15C4TnrrVdym...,audio_features,spotify:track:15C4TnrrVdym7UykxQIOTZ,0.397
4809,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,en,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,...,0.1290,-5.212,1.0,0.1630,84.448,4.0,https://api.spotify.com/v1/tracks/3zc8VZEpM1on...,audio_features,spotify:track:3zc8VZEpM1onYV4FWGdFvm,0.535


In [3]:
songs['Is hit'] = [1 if i <= 10 else 0 for i in songs['Position']]
display(songs)

,Position,Artist,Song Title,Year,JSON file,Split Names,Song Language,Spotify ID,Spotify Artist,Spotify Song,...,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,Is hit
0,2,Nat King Cole,Mona Lisa,1950,1.json,Nat King Cole,en,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa,...,-12.201,1.0,0.0371,174.578,3.0,https://api.spotify.com/v1/tracks/5dae01pKNjRQ...,audio_features,spotify:track:5dae01pKNjRQtgOeAkFzPY,0.303,1
1,5,Gary and Bing Crosby,Simple Melody,1950,4.json,Gary,en,33FdwvfiSBRbE2AQ4UehaR,Bing Crosby,Play A Simple Melody - Single Version,...,-13.755,1.0,0.0473,138.800,3.0,https://api.spotify.com/v1/tracks/33FdwvfiSBRb...,audio_features,spotify:track:33FdwvfiSBRbE2AQ4UehaR,0.829,1
2,6,Teresa Brewer,"Music, Music, Music",1950,5.json,Teresa Brewer,en,0lO5EKoz1Rb1pJoPoldE4D,Teresa Brewer,(Put Another Nickel In) Music! Music! Music!,...,-14.392,1.0,0.0398,99.136,4.0,https://api.spotify.com/v1/tracks/0lO5EKoz1Rb1...,audio_features,spotify:track:0lO5EKoz1Rb1pJoPoldE4D,0.919,1
3,7,Guy Lombardo,Third Man Theme,1950,6.json,Guy Lombardo,en,6kVfwQOc2PjVPwbk0aEmN9,Guy Lombardo,Third Man Theme,...,-10.808,1.0,0.0358,115.590,4.0,https://api.spotify.com/v1/tracks/6kVfwQOc2PjV...,audio_features,spotify:track:6kVfwQOc2PjVPwbk0aEmN9,0.536,1
4,8,Red Foley,Chattanoogie Shoe Shine Boy,1950,7.json,Red Foley,en,7Jf323ttHKUnPylFWiaGl3,Red Foley,Chattanoogie Shoe Shine Boy - 1949 Single Version,...,-15.925,1.0,0.0494,148.367,4.0,https://api.spotify.com/v1/tracks/7Jf323ttHKUn...,audio_features,spotify:track:7Jf323ttHKUnPylFWiaGl3,0.846,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4806,95,Cole Swindell,Single Saturday Night,2021,6774.json,Cole Swindell,en,1irAliF0T8sLIOPJp6n7rU,Cole Swindell,Single Saturday Night,...,-5.070,1.0,0.0473,81.949,4.0,https://api.spotify.com/v1/tracks/1irAliF0T8sL...,audio_features,spotify:track:1irAliF0T8sLIOPJp6n7rU,0.694,0
4807,96,Lainey Wilson,Things A Man Oughta Know,2021,6775.json,Lainey Wilson,en,5QS8PNEWbqTEZyQ6e9ZbJf,Lainey Wilson,Things A Man Oughta Know,...,-5.623,1.0,0.0312,139.931,4.0,https://api.spotify.com/v1/tracks/5QS8PNEWbqTE...,audio_features,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,0.397,0
4808,97,BRS Kash,Throat Baby (Go Baby),2021,6776.json,BRS Kash,en,15C4TnrrVdym7UykxQIOTZ,BRS Kash,Throat Baby (Go Baby) (with DaBaby & City Girl...,...,-8.420,0.0,0.3240,131.988,4.0,https://api.spotify.com/v1/tracks/15C4TnrrVdym...,audio_features,spotify:track:15C4TnrrVdym7UykxQIOTZ,0.397,0
4809,98,Rod Wave,Tombstone,2021,6777.json,Rod Wave,en,3zc8VZEpM1onYV4FWGdFvm,Rod Wave,Tombstone,...,-5.212,1.0,0.1630,84.448,4.0,https://api.spotify.com/v1/tracks/3zc8VZEpM1on...,audio_features,spotify:track:3zc8VZEpM1onYV4FWGdFvm,0.535,0


In [4]:
# Count hits and non-hits
songs['Is hit'].value_counts()

0    4264
1     547
Name: Is hit, dtype: int64

In [5]:
# Split data into train and test subsets
X = songs[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
y = songs['Is hit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 55)

print('Train data shape: {}'.format(X_train.shape))
print('Test data shape: {}'.format(X_test.shape))

Train data shape: (3608, 13)
Test data shape: (1203, 13)


In [6]:
# Handling imbalanced train data with smote
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

In [7]:
print(X_smote.shape, y_smote.shape)

(6378, 13) (6378,)


In [8]:
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 3)
random_forest.fit(X_smote, y_smote)
y_pred = random_forest.predict(X_test)

In [9]:
# Count predicted values
pd.Series(y_pred).value_counts()

0    1114
1      89
dtype: int64

In [10]:
print("Train accuracy: ", random_forest.score(X_smote, y_smote))
print("Test accuracy: ", random_forest.score(X_test, y_test))

Train accuracy:  0.9918469739730323
Test accuracy:  0.8478802992518704


In [11]:
# Load data from last week